# Analyse Answered Questionaires

In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema.messages import SystemMessage
import yaml
import os
import json
import pandas as pd
import glob

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)
os.environ['OPENAI_API_KEY'] = config['openai_api_key']

## Analyse Each Question From Initial Questionnaire

Individually

In [2]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(open('./persona_descriptions/analyse_each_question_from_initial_questionnaire.txt').read())
        ),
        HumanMessagePromptTemplate.from_template(
            """Can you to the best of your ability analyse the responses from this question and answer set: {response}"""
            ),
    ]
)


llm = ChatOpenAI(model='gpt-4-turbo-preview') ## make this defined ones and used again and again!!



/Users/adamfletcher/Documents/GitHub/ai_delphi_framework/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [3]:
files = glob.glob('outputs/8_formatted_questions/*.md')
for file in files:
    with open(file) as f:
        print(os.path.basename(file).replace('.md',''))
        response = f.read()
        analysed_questionaire = llm.invoke(chat_template.format_messages(response=response)).content

        with open(f"outputs/9_analysed_questions/{os.path.basename(file).replace('.md','')}.md", "w") as text_file:
            text_file.write(analysed_questionaire)

What specific best practices would you recommend for data anonymization and encryption to maintain confidentiality in handling PII and sensitive IP information within the email triage process?
What considerations would you take into account when assessing cloud vs. on-premise deployment options in relation to data security and operational efficiency for email triage?
How valuable do you find developing interfaces for departmental staff to provide feedback on email triage accuracy to improve model performance in the context of user experience?
In your opinion, how critical is ensuring the protection of Personally Identifiable Information (PII) and sensitive intellectual property (IP) within the machine learning lifecycle for email triage?
How would you suggest incorporating mechanisms for continuous learning and adaptation to address evolving departmental categorization needs as new types of emails are received?
How important is the selection of appropriate machine learning frameworks, 

## Combine Analyses into a Full Summary

In [4]:
files = glob.glob('outputs/9_analysed_questions/*.md')
text_lines = []
for file in files:
    with open(file) as f:
        question = os.path.basename(file).replace('.md','')
        # print(question)
        response = f.read()
        line = f"""## {question}

        {response}

"""
    text_lines.append(line)


## Convert Inital Analysis into Summary

In [6]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(open('./persona_descriptions/convert_initial_analysis_into_summary.txt').read())
        ),
        HumanMessagePromptTemplate.from_template(
            """Can you to the best of your ability summarise the analyses around the follow questions: {text_lines}"""
            ),
    ]
)

llm = ChatOpenAI(model='gpt-4-turbo-preview') ## make this defined ones and used again and again!!
grand_summary = llm.invoke(chat_template.format_messages(text_lines=text_lines)).content

with open("outputs/10_grand_summary.md", "w") as text_file:
    text_file.write(grand_summary)

In [ ]:
# Todo: now grab follow up questions from 9_analysis and create a new questionaire to ask the experts!